In [261]:
# import duckdb
# from pathlib import Path

# PARQUET_DIR = Path("/home/moshe/Documents/GitHub/SHM_SA_reporter/data/parquet")
# DB_FILE = PARQUET_DIR / "shm_report.duckdb"

# if DB_FILE.exists():
#     DB_FILE.unlink()

# con = duckdb.connect(str(DB_FILE))

# print("Creating:", DB_FILE)

# file_map = {
#     "2025W49_accel_all.parquet": "accel_all",
#     "2025W49_fft_hub1.parquet": "fft_hub1",
#     "2025W49_fft_hub2.parquet": "fft_hub2",
#     "2025W49_fft_hub3.parquet": "fft_hub3",
#     "2025W49_fft_hub4.parquet": "fft_hub4",
#     "2025W49_sst_hub1.parquet": "sst_hub1",
#     "2025W49_sst_hub2.parquet": "sst_hub2",
#     "2025W49_sst_hub3.parquet": "sst_hub3",
#     "2025W49_sst_hub4.parquet": "sst_hub4",
# }

# # Build tables
# for fname, table in file_map.items():
#     pf = PARQUET_DIR / fname
#     print(f"\n=== Building table '{table}' from {fname} ===")

#     con.execute(f"""
#         CREATE TABLE {table} AS 
#         SELECT * FROM parquet_scan('{pf}');
#     """)

# # 🚨 FORCE MATERIALIZATION OF THE DATABASE
# con.execute("ALTER TABLE accel_all ADD COLUMN __force_write__ INT DEFAULT 0;")
# con.execute("ALTER TABLE accel_all DROP COLUMN __force_write__;")

# con.close()

# print("✔ File exists:", DB_FILE.exists())
# print("✔ DB path:", DB_FILE)


In [262]:
import duckdb
from pathlib import Path

DB_PATH = Path("/home/moshe/Documents/GitHub/SHM_SA_reporter/data/parquet/shm_report.duckdb")

con = duckdb.connect(str(DB_PATH))

print("Connected to:", DB_PATH)
print("Tables:")
print(con.execute("SHOW TABLES").df())


Connected to: /home/moshe/Documents/GitHub/SHM_SA_reporter/data/parquet/shm_report.duckdb
Tables:
        name
0  accel_all
1   fft_hub1
2   fft_hub2
3   fft_hub3
4   fft_hub4
5   sst_hub1
6   sst_hub2
7   sst_hub3
8   sst_hub4


In [263]:
import plot_functions

# --- Auto-detect start/end datetime from the weekly DB ---
row = con.execute("""
    SELECT MIN(datetime) AS start_dt,
           MAX(datetime) AS end_dt
    FROM sst_hub1
""").fetchone()

start_dt, end_dt = row[0], row[1]

print("Detected week range:")
print("Start:", start_dt)
print("End:  ", end_dt)

# --- Run your existing matplotlib function, but do NOT save ---
plot_functions.temps_col(con, start_dt=start_dt, end_dt=end_dt, show=False, save=True)


Detected week range:
Start: 2025-12-01 00:00:15
End:   2025-12-07 23:59:34


In [270]:
import importlib
import plotly_functions

importlib.reload(plotly_functions)

plotly_functions.temps_col_plotly(con, save=True, show=False, bin_time=60)


In [269]:
import importlib
import plotly_functions

importlib.reload(plotly_functions)

plotly_functions.temps_arch_plotly(con, save=True, show=False, bin_time=60)

In [266]:
import importlib
import plotly_functions
importlib.reload(plotly_functions)

# plotly_functions.fft_with_KDE_plotly(con, sensor_id="A31", save=True)

for nmbr in range(6):
    snsr_id = (f'A3{nmbr}')
    plotly_functions.fft_with_KDE_plotly(con, sensor_id=snsr_id, show=False, save=True)


In [267]:
import importlib
import plotly_functions
importlib.reload(plotly_functions)

for nmbr in range(7, 30):
    snsr_id = (f'S{nmbr}')
    plotly_functions.strain_temp_plotly(con, sensor_id=snsr_id, show=False, save=True, bin_time=20)

# plotly_functions.strain_temp_plotly(con, sensor_id="S13", save=True, bin_time=20)


In [268]:
import importlib
import plotly_functions
importlib.reload(plotly_functions)

for nmbr in range(6):
    snsr_id = (f'A3{nmbr}')
    plotly_functions.accel_v_daily_grid_plotly(con, sensor_id=snsr_id, show=False, save=True)


# plotly_functions.accel_v_daily_grid_plotly(con, "A31", save=True)
